# 🎯 ML Priority Scoring System - Training & Evaluation

This notebook trains and evaluates three ML models (Random Forest, XGBoost, LightGBM) to predict:
1. **Modernization Score** - Probability equipment needs modernization
2. **OEM Score** - Probability equipment needs total replacement
3. **Maintenance Score** - Probability equipment needs lifecycle services

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb

from app.services.ml_priority_service import MLPriorityService
from app.core.config import settings

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports successful")

## 1. Connect to Database

In [ ]:
# Connect to DuckDB
db_path = settings.DB_PATH
conn = duckdb.connect(str(db_path))

# Check available tables
tables = conn.execute("SHOW TABLES").df()
print("Available tables:")
print(tables)

## 2. Initialize ML Service

In [ ]:
# Initialize service with database connection
ml_service = MLPriorityService(db_conn=conn)
print("✅ ML Service initialized")

## 3. Extract & Explore Training Data

In [ ]:
# Extract training data
df = ml_service.extract_training_data()

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Visualize equipment age distribution
plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
df['equipment_age'].hist(bins=30, edgecolor='black')
plt.xlabel('Equipment Age (years)')
plt.ylabel('Count')
plt.title('Equipment Age Distribution')

plt.subplot(1, 3, 2)
df['age_category'].value_counts().plot(kind='bar')
plt.xlabel('Age Category')
plt.ylabel('Count')
plt.title('Equipment by Age Category')
plt.xticks(rotation=45)

plt.subplot(1, 3, 3)
df.groupby('equipment_type_clean')['equipment_age'].mean().sort_values(ascending=False).head(10).plot(kind='barh')
plt.xlabel('Average Age (years)')
plt.title('Top 10 Equipment Types by Avg Age')

plt.tight_layout()
plt.show()

## 4. Generate Training Labels

In [ ]:
# Generate labels
df = ml_service.generate_labels(df)

# Visualize label distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (target, ax) in enumerate(zip(['modernization_target', 'oem_target', 'maintenance_target'], axes)):
    df[target].value_counts().plot(kind='bar', ax=ax, color=['lightcoral', 'lightblue'])
    ax.set_title(f"{target.replace('_target', '').title()} Labels")
    ax.set_xlabel('Label')
    ax.set_ylabel('Count')
    ax.set_xticklabels(['No', 'Yes'], rotation=0)

plt.tight_layout()
plt.show()

## 5. Train All Models

In [ ]:
# Train models for all three targets
ml_service.train_all_models()

## 6. Model Performance Comparison

In [ ]:
# Collect performance metrics
performance_data = []

for target_type in ['modernization', 'oem', 'maintenance']:
    for model_name, model_data in ml_service.models[target_type].items():
        metrics = model_data['metrics']
        performance_data.append({
            'Target': target_type.title(),
            'Model': metrics['model_name'],
            'AUC-ROC': metrics['auc_roc'],
            'AUC-PR': metrics['auc_pr'],
            'Accuracy': metrics['accuracy']
        })

perf_df = pd.DataFrame(performance_data)
print("\n📊 Model Performance Summary:")
print(perf_df.to_string(index=False))

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, metric in enumerate(['AUC-ROC', 'AUC-PR', 'Accuracy']):
    pivot = perf_df.pivot(index='Model', columns='Target', values=metric)
    pivot.plot(kind='bar', ax=axes[idx], width=0.8)
    axes[idx].set_title(f'{metric} Comparison')
    axes[idx].set_ylabel(metric)
    axes[idx].set_ylim([0, 1])
    axes[idx].legend(title='Target Type')
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Generate Priority List

In [ ]:
# Generate predictions for all equipment
priority_df = ml_service.predict_priorities(df)

# Display top priorities
print("\n🎯 TOP 20 PRIORITY EQUIPMENT FOR MODERNIZATION:")
priority_cols = [
    'company_internal', 'equipment_type_clean', 'equipment_age',
    'modernization_score', 'oem_score', 'maintenance_score', 'priority_rank'
]
print(priority_df[priority_cols].head(20).to_string(index=False))

In [ ]:
# Visualize score distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, score_col in enumerate(['modernization_score', 'oem_score', 'maintenance_score']):
    axes[idx].hist(priority_df[score_col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].axvline(priority_df[score_col].median(), color='red', linestyle='--', label=f'Median: {priority_df[score_col].median():.1f}')
    axes[idx].set_xlabel('Score')
    axes[idx].set_ylabel('Count')
    axes[idx].set_title(f"{score_col.replace('_', ' ').title()} Distribution")
    axes[idx].legend()

plt.tight_layout()
plt.show()

## 8. Export Priority List

In [ ]:
# Export to CSV
output_file = '../data/priority_list.csv'
priority_df[priority_cols].to_csv(output_file, index=False)
print(f"✅ Priority list exported to: {output_file}")

## 9. Save to DuckDB

In [ ]:
# Save priority scores to database
conn.execute("DROP TABLE IF EXISTS equipment_priorities")
conn.execute("CREATE TABLE equipment_priorities AS SELECT * FROM priority_df")
print("✅ Priority scores saved to DuckDB table: equipment_priorities")